Vocab

In [24]:
import pandas as pd
import numpy as  np
from gensim.models import Word2Vec
import gensim.downloader 
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import TensorDataset, random_split
from torch import nn
import pickle
import torch.nn.functional as F

In [8]:
df_title = pd.read_pickle('./data/clean_title.pkl')
df_text = pd.read_pickle('./data/clean_text.pkl')
print(df_title.columns)
print(df_text.columns)
print(df_title.head())
print(df_text.head())

Index(['index', 'title', 'class'], dtype='object')
Index(['index', 'text', 'class'], dtype='object')
   index                                              title  class
0      0  [law, enforcement, high, alert, following, thr...      1
2      2  [unbelievable, obama, attorney, general, says,...      1
3      3  [bobby, jindal, raised, hindu, uses, story, ch...      0
4      4  [satan, 2, russia, unvelis, image, terrifying,...      1
5      5  [time, christian, group, sues, amazon, splc, d...      1
   index                                               text  class
0      0  [comment, expected, barack, obama, members, fy...      1
1      1                    [post, votes, hillary, already]      1
2      2  [demonstrators, gathered, last, night, exercis...      1
3      3  [dozen, politically, active, pastors, came, pr...      0
4      4  [rs, 28, sarmat, missile, dubbed, satan, 2, re...      1


In [14]:
model_text = Word2Vec( pd.concat([df_text["text"] , df_title["title"]],ignore_index=True) , vector_size=300)
print("text: {0}".format(model_text.corpus_total_words))
model_text.wv.save_word2vec_format("./data/model_text")

text: 23187031


In [16]:
pretrained_model = gensim.downloader.load("word2vec-google-news-300")
pretrained_model.vectors_lockf = np.ones(len(pretrained_model), dtype=np.float32)
merged_keyed_vectors = pretrained_model.intersect_word2vec_format(
    "./data/model_text", binary=True, lockf=1.0
)

In [21]:
# test with smaller amount of data
FRACTION = 0.01
size = int(df_text.shape[0] * FRACTION)
print(size)


# make tensors
def text_to_tensor(text_list, merged_keyed_vectors):
    vectors = [merged_keyed_vectors.wv[word] for word in text_list if word in merged_keyed_vectors.wv]
    return torch.tensor(vectors)


def stack_tensor(df: pd.DataFrame, col_name, word_model):
    tensor_ls = []
    label_ls = []
    for row in df.itertuples(index=False):
        # print(row[df.columns.get_loc('class')])
        # print(row[df.columns.get_loc(col_name)])
        label_ls.append(row[df.columns.get_loc("class")])
        tensor_ls.append(text_to_tensor(row[df.columns.get_loc(col_name)], word_model))
    padded_ls = pad_sequence(tensor_ls, batch_first=True)
    return torch.tensor(label_ls), torch.stack(tuple(padded_ls))


text_labels_tensor, text_tensor = stack_tensor(
    df_text.iloc[:size, :], 
    col_name="text", 
    word_model=model_text
)
print(text_tensor.size())
print(text_labels_tensor.size())
text_dataset = TensorDataset(text_tensor, text_labels_tensor)

713


/var/folders/nf/g_pc_4zd2sx16tjwwb4r4fhm0000gn/T/ipykernel_72111/2303759884.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:278.)
  return torch.tensor(vectors)


torch.Size([713, 12203, 300])
torch.Size([713])


In [22]:
def split(tensor_dataset, test_split=0.2):
    test_size = int(len(tensor_dataset) * test_split)
    train_size = int(len(tensor_dataset) - test_size)
    print(test_size)
    print(train_size)
    train_data, test_data = random_split(tensor_dataset, [train_size, test_size])
    return train_data, test_data

train_text_tensor, test_text_tensor = split(text_dataset,0.2)

with open("./data/train_text","wb") as f:
    pickle.dump(train_text_tensor,f)
with open("./data/test_text","wb") as f:
    pickle.dump(train_text_tensor,f)


142
571
